# 基于期货文档的方案

初步结论：

- DocumentSummaryIndex
    - 适合文档中找具体答案
    - 不适合做预测，因为不能理解完整内容
- 不适合期货文档做预测的事情

## 准备

In [1]:
%%time

INDEX_PATH="simple-index"

CPU times: user 1 µs, sys: 0 ns, total: 1 µs
Wall time: 3.1 µs


In [2]:
%%time
!rm -rf $INDEX_PATH

CPU times: user 9.7 ms, sys: 257 µs, total: 9.96 ms
Wall time: 108 ms


In [3]:
%%time
%%capture

from llama_index.core import SimpleDirectoryReader

# file_metadata = lambda x: {"filename": x}

documents=SimpleDirectoryReader(input_dir="./simple-data").load_data(num_workers=4)
for document in documents:
    document.doc_id=document.metadata['file_name']

Ignoring wrong pointing object 6 0 (offset 0)
Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 10 0 (offset 0)
Ignoring wrong pointing object 12 0 (offset 0)
Ignoring wrong pointing object 14 0 (offset 0)
Ignoring wrong pointing object 16 0 (offset 0)
Ignoring wrong pointing object 19 0 (offset 0)
Ignoring wrong pointing object 21 0 (offset 0)
Ignoring wrong pointing object 23 0 (offset 0)
Ignoring wrong pointing object 25 0 (offset 0)
Ignoring wrong pointing object 27 0 (offset 0)
Ignoring wrong pointing object 29 0 (offset 0)
Ignoring wrong pointing object 31 0 (offset 0)
Ignoring wrong pointing object 33 0 (offset 0)
Ignoring wrong pointing object 35 0 (offset 0)
Ignoring wrong pointing object 37 0 (offset 0)
Ignoring wrong pointing object 45 0 (offset 0)
Ignoring wrong pointing object 59 0 (offset 0)
Ignoring wrong pointing object 61 0 (offset 0)
Ignoring wrong pointing object 63 0 (offset 0)
Ignoring wrong pointing object 66 0 (offset 0)
Ignoring wrong 

CPU times: user 2.78 s, sys: 335 ms, total: 3.12 s
Wall time: 17.1 s


In [4]:
%%time

import nest_asyncio
nest_asyncio.apply()

CPU times: user 1.18 ms, sys: 100 µs, total: 1.29 ms
Wall time: 1.01 ms


In [5]:
%%time

# 加载llm和embeddings
%run ../utils2.py

from llama_index.core import Settings

# Settings.llm=get_llm("gpt-3.5-turbo")
Settings.llm=get_llm()
Settings.embed_model=get_embedding()

CPU times: user 682 ms, sys: 31 ms, total: 713 ms
Wall time: 714 ms


In [6]:
from llama_index.core.node_parser import SentenceSplitter

splitter = SentenceSplitter(chunk_size=1024)

In [7]:
%%time

from llama_index.core import get_response_synthesizer
from llama_index.core import DocumentSummaryIndex

response_synthesizer = get_response_synthesizer(
    response_mode="tree_summarize", use_async=True
)

doc_summary_index = DocumentSummaryIndex.from_documents(
    documents,
    transformations=[splitter],
    response_synthesizer=response_synthesizer,
    show_progress=True,
)

Parsing nodes:   0%|          | 0/517 [00:00<?, ?it/s]

Summarizing documents:   0%|          | 0/32 [00:00<?, ?it/s]

current doc id: 20240623-【外汇周报】日本被列汇率操纵监测名单，瑞士央行进行第二次降息.pdf
current doc id: 240622【纸浆周报】进口端支撑犹在 供需双弱博弈加剧.pdf
current doc id: 《投产预期临近，注意交易节奏》中信建投期货尿素周报.pdf
current doc id: 【双焦周报】等待变量-2024-06-22.pdf
current doc id: 【工业硅周报】供应持续增加，价格弱势下行2024-06-23.pdf
current doc id: 【建投专题】乙烯原料轻质化背景下，丁二烯产能增速放缓20240622.pdf
current doc id: 【建投专题】梳理PX供需现状，09合约难现交割行情20240622.pdf
current doc id: 【建投专题】煤：国内宏观现实的“金丝雀”及商品传导.pdf
current doc id: 【建投周报】PX-PTA-MEG-PF估值回归等驱动，矛盾集中于需求端20240623.pdf
current doc id: 【建投碳学】国外简评：若燃油车禁令如期撤除，欧盟替代方案几何？-20240621.pdf
current doc id: 【橡胶周报】回归基本面定价，等待新驱动出现20240622.pdf
current doc id: 【油脂周报】美加种植面积或调整，棕榈油累库节奏偏慢 2024-06-23.pdf
current doc id: 【糖史演义】原糖关注巴西供应链风险，国内近月向下空间有限.pdf
current doc id: 【花生周报】关注出苗期天气变化情况20240622.pdf
current doc id: 【贵金属周报】市场情绪快速变化，贵金属宽幅震荡2024-06-23.pdf
current doc id: 【钢材周报】铁水上行， 钢价重心下移20240622.pdf
current doc id: 【铁矿&废钢周报】利空逐渐积累，铁矿关注800关口20240623.pdf
current doc id: 【铜期货期权周报】经济担忧加剧，铜价跌势未停2024-06-23.pdf
current doc id: 【铝周报】宏观情绪偏空，沪铝震荡偏弱 2024-6-23.pdf
current doc i

Generating embeddings:   0%|          | 0/32 [00:00<?, ?it/s]

CPU times: user 2.54 s, sys: 35.3 ms, total: 2.58 s
Wall time: 20min 24s


In [8]:
%%time

doc_summary_index.storage_context.persist("simple-index")

CPU times: user 112 ms, sys: 12 ms, total: 124 ms
Wall time: 124 ms


In [9]:
%%time

from llama_index.core import load_index_from_storage
from llama_index.core import StorageContext

# rebuild storage context
storage_context = StorageContext.from_defaults(persist_dir="simple-index")
doc_summary_index = load_index_from_storage(storage_context)

CPU times: user 202 ms, sys: 3.67 ms, total: 206 ms
Wall time: 205 ms


In [11]:
%%time

query_engine = doc_summary_index.as_query_engine(
    response_mode="tree_summarize", use_async=True
)

response = query_engine.query("帮我梳理核心内容，帮我预测未来一周的走势，计算每一项分项指标数据，基于分项指标计算综合指标。数据在-100%到100%中间量化展示。")
response

CPU times: user 190 ms, sys: 12 ms, total: 202 ms
Wall time: 2min 42s


Response(response='### 核心内容梳理与预测\n\n#### 天气与土壤墒情影响分析：\n\n- **高温天气**：预计未来一周华北南部、黄淮地区将经历持续的高温晴热少雨，这将加剧土壤水分蒸发和失水情况。河北、山西、山东等地区的土壤缺水状况将持续恶化。\n- **干旱程度评估**：\n  - 河北保定、石家庄，山西阳泉、晋中、太原、临汾，陕西延安东部等地区面临重度缺墒问题。\n  - 河北中南部、山西中南部、陕西中部、山东中部存在轻度缺墒情况。\n\n#### 替代品分析与市场动态：\n\n- **小麦基本面**：通过比较小麦和玉米在不同时间点的价格差异，可以预测市场对替代品的选择。高温天气可能导致小麦需求增加，从而影响其价格。\n- **面粉企业开机率变化**：根据收集的面粉企业的生产记录和计划调整数据，计算开机率的变化百分比。\n\n#### 海外市场关注点：\n\n- **美国燃料乙醇基本面**：分析日需求量及其对全球市场的潜在影响。\n\n### 预测未来一周走势：\n\n1. **天气因素**：\n   - 高温和少雨可能导致土壤进一步失水，增加干旱程度。这将直接影响作物生长、产量预期及农产品供需平衡。\n2. **替代品分析**：\n   - 小麦与玉米价格差异及面粉企业开机率的变化可能影响市场对替代品的选择，特别是需求端的调整或生产活动的变动。\n3. **海外因素**：\n   - 美国燃料乙醇市场的动态通过能源和生物燃料供应链间接影响全球粮食市场。\n\n### 综合指标计算与量化展示：\n\n#### 1. 天气指标（-100%到100%）：\n\n- 使用干旱程度评估数据，重度缺墒地区为-80%，轻度缺墒地区为-20%。通过颜色编码或图表显示不同地区的干旱等级。\n\n#### 2. 替代品指标（百分比变化）：\n\n- 小麦与玉米的比价关系计算，通过折线图展示价格走势及比值变化。\n- 利润差异分析，以柱状图表示小麦和玉米在不同时间点上的利润对比。\n\n#### 3. 海外市场指标（指数变化）：\n\n- 美国燃料乙醇日需求量与产量比例的动态变化，通过折线图展示供需平衡状态。\n- 出口活动强度，结合出口销售与检验量数据，使用柱状图或热力图显示全球对美国玉米的需求趋势。\n\n### 注意事项：\n\n1. **数据解读

In [13]:
print(response.response)

### 核心内容梳理与预测

#### 天气与土壤墒情影响分析：

- **高温天气**：预计未来一周华北南部、黄淮地区将经历持续的高温晴热少雨，这将加剧土壤水分蒸发和失水情况。河北、山西、山东等地区的土壤缺水状况将持续恶化。
- **干旱程度评估**：
  - 河北保定、石家庄，山西阳泉、晋中、太原、临汾，陕西延安东部等地区面临重度缺墒问题。
  - 河北中南部、山西中南部、陕西中部、山东中部存在轻度缺墒情况。

#### 替代品分析与市场动态：

- **小麦基本面**：通过比较小麦和玉米在不同时间点的价格差异，可以预测市场对替代品的选择。高温天气可能导致小麦需求增加，从而影响其价格。
- **面粉企业开机率变化**：根据收集的面粉企业的生产记录和计划调整数据，计算开机率的变化百分比。

#### 海外市场关注点：

- **美国燃料乙醇基本面**：分析日需求量及其对全球市场的潜在影响。

### 预测未来一周走势：

1. **天气因素**：
   - 高温和少雨可能导致土壤进一步失水，增加干旱程度。这将直接影响作物生长、产量预期及农产品供需平衡。
2. **替代品分析**：
   - 小麦与玉米价格差异及面粉企业开机率的变化可能影响市场对替代品的选择，特别是需求端的调整或生产活动的变动。
3. **海外因素**：
   - 美国燃料乙醇市场的动态通过能源和生物燃料供应链间接影响全球粮食市场。

### 综合指标计算与量化展示：

#### 1. 天气指标（-100%到100%）：

- 使用干旱程度评估数据，重度缺墒地区为-80%，轻度缺墒地区为-20%。通过颜色编码或图表显示不同地区的干旱等级。

#### 2. 替代品指标（百分比变化）：

- 小麦与玉米的比价关系计算，通过折线图展示价格走势及比值变化。
- 利润差异分析，以柱状图表示小麦和玉米在不同时间点上的利润对比。

#### 3. 海外市场指标（指数变化）：

- 美国燃料乙醇日需求量与产量比例的动态变化，通过折线图展示供需平衡状态。
- 出口活动强度，结合出口销售与检验量数据，使用柱状图或热力图显示全球对美国玉米的需求趋势。

### 注意事项：

1. **数据解读**：在分析各项指标时，需关注具体数值、时间点及年份间的比较，以准确理解市场动态。
2. **预测局限性**：任何预测都存在不确

In [15]:
from IPython.display import display, Markdown

display(Markdown(f'{response.response}'))

### 核心内容梳理与预测

#### 天气与土壤墒情影响分析：

- **高温天气**：预计未来一周华北南部、黄淮地区将经历持续的高温晴热少雨，这将加剧土壤水分蒸发和失水情况。河北、山西、山东等地区的土壤缺水状况将持续恶化。
- **干旱程度评估**：
  - 河北保定、石家庄，山西阳泉、晋中、太原、临汾，陕西延安东部等地区面临重度缺墒问题。
  - 河北中南部、山西中南部、陕西中部、山东中部存在轻度缺墒情况。

#### 替代品分析与市场动态：

- **小麦基本面**：通过比较小麦和玉米在不同时间点的价格差异，可以预测市场对替代品的选择。高温天气可能导致小麦需求增加，从而影响其价格。
- **面粉企业开机率变化**：根据收集的面粉企业的生产记录和计划调整数据，计算开机率的变化百分比。

#### 海外市场关注点：

- **美国燃料乙醇基本面**：分析日需求量及其对全球市场的潜在影响。

### 预测未来一周走势：

1. **天气因素**：
   - 高温和少雨可能导致土壤进一步失水，增加干旱程度。这将直接影响作物生长、产量预期及农产品供需平衡。
2. **替代品分析**：
   - 小麦与玉米价格差异及面粉企业开机率的变化可能影响市场对替代品的选择，特别是需求端的调整或生产活动的变动。
3. **海外因素**：
   - 美国燃料乙醇市场的动态通过能源和生物燃料供应链间接影响全球粮食市场。

### 综合指标计算与量化展示：

#### 1. 天气指标（-100%到100%）：

- 使用干旱程度评估数据，重度缺墒地区为-80%，轻度缺墒地区为-20%。通过颜色编码或图表显示不同地区的干旱等级。

#### 2. 替代品指标（百分比变化）：

- 小麦与玉米的比价关系计算，通过折线图展示价格走势及比值变化。
- 利润差异分析，以柱状图表示小麦和玉米在不同时间点上的利润对比。

#### 3. 海外市场指标（指数变化）：

- 美国燃料乙醇日需求量与产量比例的动态变化，通过折线图展示供需平衡状态。
- 出口活动强度，结合出口销售与检验量数据，使用柱状图或热力图显示全球对美国玉米的需求趋势。

### 注意事项：

1. **数据解读**：在分析各项指标时，需关注具体数值、时间点及年份间的比较，以准确理解市场动态。
2. **预测局限性**：任何预测都存在不确定性，实际走势可能受到突发事件、政策变化等不可预测因素的影响。

In [16]:
%%time

query_engine = doc_summary_index.as_query_engine(
    response_mode="tree_summarize", 
    use_async=True,
    streaming=True,
)

response = query_engine.query("帮我预测未来一周的纯碱玻璃走势，计算每一项分项指标数据，基于分项指标计算综合指标。数据在-100%到100%中间量化展示。")
response.print_response_stream()

要准确预测未来一周的纯碱和玻璃走势并计算各项分项指标数据，需要遵循以下步骤：

### 收集市场信息

1. **供需情况**：了解当前及预期的工业生产活动水平、建筑和制造等行业的需求情况。这包括新生产线的建设和现有生产线的运行状态。
2. **原材料成本**：关注纯碱和玻璃的主要原料价格变动，如盐、石灰石等的价格走势，以及政策因素对这些原材料市场的影响。
3. **政策动态**：收集政府对于工业生产的限制或鼓励措施、环保政策等相关信息，评估其对行业发展的潜在影响。
4. **国际市场动态**：考虑纯碱和玻璃作为大宗产品的国际供需情况，分析可能的全球价格波动。

### 分析历史数据

1. **价格变动趋势**：回顾过去一段时间内纯碱和玻璃的价格走势，识别短期和长期的趋势。
2. **成交量与持仓量**：分析市场交易活动的活跃程度以及投资者对这些商品的态度变化。
3. **技术指标**：应用如移动平均线、相对强弱指数（RSI）、布林带等技术工具来评估当前市场的状态和潜在变动。

### 考虑外部因素

1. **宏观经济环境**：考虑GDP增长、通货膨胀率等因素对工业生产的影响。
2. **政策影响**：分析政府的经济政策、环保法规等如何影响纯碱和玻璃的生产和消费。
3. **成本变化**：评估原材料价格波动、运输成本、能源成本等对行业成本结构的影响。

### 综合因素考量

1. **供需平衡**：结合市场信息和历史数据，评估当前及预期的供需情况。
2. **技术分析与基本面分析**：将技术指标与行业基本面相结合，形成综合预测模型。
3. **风险评估**：考虑潜在的风险因素，如政策变动、自然灾害等对价格波动的影响。

### 数据量化展示

- 使用统计方法或专业软件计算各项分项指标数据，并在-100%到100%的范围内进行量化展示。例如：
  - 如果预测纯碱价格将上涨2%，则表示为+2%。
  - 如果预计消费量会减少3%，则表示为-3%。

### 结论

由于缺乏具体的数据和分析框架，实际操作需要依赖专业的市场分析师或金融工具来处理复杂数据，并结合最新的行业动态进行综合评估。在进行任何投资决策前，请咨询专业意见并考虑个人风险承受能力。CPU times: user 852 ms, sys: 56.4 ms, total: 909 ms
Wall ti

In [18]:
display(Markdown(f'{response}'))

要准确预测未来一周的纯碱和玻璃走势并计算各项分项指标数据，需要遵循以下步骤：

### 收集市场信息

1. **供需情况**：了解当前及预期的工业生产活动水平、建筑和制造等行业的需求情况。这包括新生产线的建设和现有生产线的运行状态。
2. **原材料成本**：关注纯碱和玻璃的主要原料价格变动，如盐、石灰石等的价格走势，以及政策因素对这些原材料市场的影响。
3. **政策动态**：收集政府对于工业生产的限制或鼓励措施、环保政策等相关信息，评估其对行业发展的潜在影响。
4. **国际市场动态**：考虑纯碱和玻璃作为大宗产品的国际供需情况，分析可能的全球价格波动。

### 分析历史数据

1. **价格变动趋势**：回顾过去一段时间内纯碱和玻璃的价格走势，识别短期和长期的趋势。
2. **成交量与持仓量**：分析市场交易活动的活跃程度以及投资者对这些商品的态度变化。
3. **技术指标**：应用如移动平均线、相对强弱指数（RSI）、布林带等技术工具来评估当前市场的状态和潜在变动。

### 考虑外部因素

1. **宏观经济环境**：考虑GDP增长、通货膨胀率等因素对工业生产的影响。
2. **政策影响**：分析政府的经济政策、环保法规等如何影响纯碱和玻璃的生产和消费。
3. **成本变化**：评估原材料价格波动、运输成本、能源成本等对行业成本结构的影响。

### 综合因素考量

1. **供需平衡**：结合市场信息和历史数据，评估当前及预期的供需情况。
2. **技术分析与基本面分析**：将技术指标与行业基本面相结合，形成综合预测模型。
3. **风险评估**：考虑潜在的风险因素，如政策变动、自然灾害等对价格波动的影响。

### 数据量化展示

- 使用统计方法或专业软件计算各项分项指标数据，并在-100%到100%的范围内进行量化展示。例如：
  - 如果预测纯碱价格将上涨2%，则表示为+2%。
  - 如果预计消费量会减少3%，则表示为-3%。

### 结论

由于缺乏具体的数据和分析框架，实际操作需要依赖专业的市场分析师或金融工具来处理复杂数据，并结合最新的行业动态进行综合评估。在进行任何投资决策前，请咨询专业意见并考虑个人风险承受能力。

In [20]:
%%time

query_engine = doc_summary_index.as_query_engine(
    response_mode="tree_summarize", 
    use_async=True,
)

response = query_engine.query("根据期货纯碱玻璃周度报告，预测未来一周的纯碱玻璃走势")
display(Markdown(f'{response.response}'))

根据提供的信息，短期内纯碱玻璃市场预计会呈现偏弱震荡的走势。主要逻辑如下：

1. **供应**：本周纯碱供应环比小幅增加，检修计划减少，产量继续增长。下周检修计划变动不大，预计产量将稳定或略有下降。这表明供应端对价格形成一定的利空影响。

2. **需求**：本周纯碱需求整体持稳。近期下游需求表现平稳，氧化铝产量略有下降，粘胶短纤、纸浆等下游开工率有所上升。然而，近期现货价格整体稳定，下游采购积极性一般。这表明需求端对价格的影响较为平衡或略显疲软。

3. **库存**：本周纯碱库存略有增加。液碱库存环比增加约0.6万吨，其中西北、华北、西南地区库存上升，其余地区库存下降。预计下周库存将窄幅波动。库存的增加可能表明市场供应相对充足，需求增长有限。

4. **价格与期货走势**：
   - 本周纯碱期货价格小幅下跌，9-1价差基本持稳。
   - 现货方面，32%液碱价格多数持稳，山东、江苏和浙江的现货价格分别为2438元/吨、2750元/吨和3281元/吨。周度涨跌幅在0.00%-2.33%之间。
   - 基差方面，截至本周五，山东烧碱现货价格与主力合约的基差略有走强。

基于上述分析，短期内纯碱玻璃市场预计会维持偏弱震荡的走势。期货价格可能继续承压，下方关注2650附近的支撑位。中期来看，考虑到纯碱9月份合约为传统旺季合约，下半年估值存在修复空间。如果SH2409合约大幅回调，则可关注做多机会，下方关注2550-2600附近的支持。

需要注意的风险包括：检修超预期、下游需求超预期等可能影响市场供需平衡的因素。

CPU times: user 57.2 ms, sys: 22 µs, total: 57.3 ms
Wall time: 32.3 s


In [21]:
%%time

query_engine = doc_summary_index.as_query_engine(
    response_mode="tree_summarize", 
    use_async=True,
)

response = query_engine.query("根据期货纯碱玻璃周度报告，预测未来一周的纯碱玻璃走势。计算每一项分项指标数据，基于分项指标计算综合指标。数据在-100%到100%中间量化展示。")
display(Markdown(f'{response.response}'))

为了根据期货纯碱和玻璃的周度报告预测未来一周的走势，并对各项分项指标进行量化展示，我们可以遵循以下步骤：

### 步骤一：收集关键指标数据

首先，从周度报告中收集以下几个关键指标：
1. **供需平衡**：了解当前市场的供应量与需求量之间的关系。
2. **价格变动**：分析过去一周内纯碱和玻璃的价格波动情况。
3. **成本因素**：包括原材料（如盐、能源）的成本变化，以及生产成本的总体趋势。
4. **政策影响**：近期或预期中的政策调整对市场可能产生的影响。
5. **库存水平**：当前市场的库存量及其变动情况。

### 步骤二：分析数据

1. **供需平衡分析**：评估供应是否充足、需求是否有增长或下降的趋势。高需求低供应可能导致价格上涨，反之则价格下跌。
2. **价格趋势分析**：通过图表和移动平均线等技术工具，观察价格的短期波动模式。
3. **成本敏感性**：分析原材料成本变动对纯碱和玻璃生产成本的影响，并预测可能的价格调整。
4. **政策影响评估**：考虑近期或预期中的政策变化如何影响市场情绪和投资决策。
5. **库存管理**：高库存可能导致价格下跌，低库存则可能推动价格上涨。

### 步骤三：综合指标计算

1. **构建模型**：基于上述分析结果，可以构建一个综合指标来量化预测。例如，可以将供需平衡、价格趋势、成本敏感性、政策影响和库存管理等因素的权重加权平均。
2. **量化展示**：
   - **供需平衡**（-100%到+100%）：高需求低供应为+100%，低需求高供应为-100%。
   - **价格趋势**（-100%到+100%）：价格上涨为+100%，价格下跌为-100%。
   - **成本影响**（-100%到+100%）：成本上升对市场不利，为-100%；成本下降有利，为+100%。
   - **政策影响**（-100%到+100%）：正面政策为+100%，负面或中性政策为0%。
   - **库存管理**（-100%到+100%）：低库存为+100%，高库存为-100%。

### 步骤四：综合指标计算

假设每个因素的权重分别为：
- 供需平衡：30%
- 价格趋势：25%
- 成本影响：20%
- 政策影响：15%
- 库存管理：10%

将上述量化值乘以各自权重后加总，可以得到一个综合指标。例如：

\[ \text{综合指标} = (-100\% \times 30\%) + (+100\% \times 25\%) + (-100\% \times 20\%) + (0\% \times 15\%) + (+100\% \times 10\%) \]

\[ = -30 + 25 - 20 + 0 + 10 = -15\% \]

### 步骤五：预测与建议

综合指标为-15%，表明市场整体趋势可能偏弱，价格有下跌的风险。然而，这只是一个简化模型的预测结果，并且实际走势受到更多因素的影响，包括未提及的全球市场动态、突发事件等。

### 结论：

通过上述步骤，我们可以基于周度报告中的数据进行量化分析和预测。但请注意，期货市场的波动性高，任何预测都存在不确定性，建议结合实时信息和专业分析进行决策。

CPU times: user 137 ms, sys: 3.9 ms, total: 141 ms
Wall time: 1min 44s
